In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd

#datasetleri yüklüyoruz
df_train=pd.read_csv("/content/drive/MyDrive/kolektif_2/turkish-sentiment/train.csv",index_col=[0],encoding="windows-1252")
df_test=pd.read_csv("/content/drive/MyDrive/kolektif_2/turkish-sentiment/test.csv",index_col=[0],encoding="windows-1252")
#df_test = pd.read_csv("/content/drive/MyDrive/kolektif_2/dataset/magaza_yorumlari_duygu_analizi.csv", encoding = "utf-16")
#df_test = pd.read_csv("/content/drive/MyDrive/kolektif_2/train.csv",index_col=[0],encoding="windows-1252")

In [3]:
import torch
# cuda varsa gpu yoksa cpu kullanmak için cihazı ayarlayan kod hücresi
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

We will use the GPU: Tesla V100-SXM2-16GB


In [4]:
#test veri seti için zero-shot a sokulmak üzere metin ve etiketlerin alınması ardından ilk 1000inin alınması
comments = df_test.comment.values

labels_test = df_test.Label.values

comments = comments[:1000]
labels_test = labels_test[:1000]

In [5]:
comments, df_test.Label.values

(array(['arkadaslar film bence cok güzel su anda gidilebilecek en iyi film izlemenizi tavsiye ederim.. \n',
        'mükemmel bir film ve sonu enfes. mutlaka izleyin, pisman olmazsiniz. rejimler ne olursa olsun, olan insana ve insanin duygularina oluyor.. \n',
        'epey bi uzak durun diyorum..bu ne ya iyice cilkini çikarmislar..adam yaratigi yaratik kadini ...nasil bir duyguyla yazmislarsa senaryoyu artik..... \n',
        'sürükleyici bir aksiyon, özellikle sonu çok sasirtici ve acikliydi.. \n',
        'hayatimda izledigim en berbat filmdi gerçekten tamamen zaman kaybi...oyunculuk,müzik,görüntüler hersey çok kötüydü...kimseye tavsiye etmem.... \n',
        "son zamanlarda gösterimde olan en ii filmdi bence.konusu siradan olabilir ama will smith'in oyunculukta mükemmel derecedeki basarisi ve vampir animasyonlar siradanliktan uzaklastirmis.kesinlikle sinemada izlenmesi gerek,çünkü ilk 60 dk temposu düsük bir film.. \n",
        'gerek kurgusu gerek seneryosu,soundtrack ve oyuncular

In [6]:
from transformers import AutoTokenizer, AutoModel
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import pickle
from transformers import AutoModelForSequenceClassification, AdamW, AutoConfig

from transformers import AutoTokenizer, AutoModel
from torch.nn import functional as F

accuracy_list= []
model_predicts = {}
#her 5 model için iterasyon kuruldu  , 'ytu-ce-cosmos/turkish-small-bert-uncased', 'ytu-ce-cosmos/turkish-tiny-bert-uncased', 'ytu-ce-cosmos/turkish-mini-bert-uncased','ytu-ce-cosmos/turkish-medium-bert-uncased' ]
models= ['dbmdz/bert-base-turkish-cased']
for model_name in models:
    #tokenizer ve model burada tanımlanıyor
    tokenizer = AutoTokenizer.from_pretrained(model_name,do_lower_case=True)
    model = AutoModel.from_pretrained(model_name, num_labels=2)
    predict_list = []
    #commentsin içerisindeki her satır için vektörler çıkartılıp cosine uzaklık predict_listin içerisine yazılıyor ardından bu liste model_predictsin içerisine yazılıyor. Ayrıca Accuracyler de Accuracy_list e yazılıyor
    for comment in comments:
        sentence = comment
        labels = ["bu bir olumlu","bu bir olumsuz"]


        # run inputs through model and mean-pool over the sequence
        # dimension to get sequence-level representations
        inputs = tokenizer.batch_encode_plus([sentence] + labels,
                                            return_tensors='pt',
                                            pad_to_max_length=True)

        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        output = model(input_ids, attention_mask=attention_mask)[0]
        sentence_rep = output[:1].mean(dim=1)
        label_reps = output[1:].mean(dim=1)

        # now find the labels with the highest cosine similarities to
        # the sentence
        similarities = F.cosine_similarity(sentence_rep, label_reps)
        #closest en yakın olan labelı önce yazıyor closest[0] en yakını vermiş oluyor
        closest = similarities.argsort(descending=True)
        predict_list.append(closest[0])
 #       print(closest)
    #model accuracyleri yazılır.
    from sklearn.metrics import accuracy_score
    accuracy_score =accuracy_score(labels_test, predict_list)
    accuracy_list.append(accuracy_score)
    print(f"Accuracy of BERT ({model_name}) is:",accuracy_score)

    from sklearn.metrics import classification_report
    print(classification_report(labels_test, predict_list))

    model_predicts[model_name]=predict_list

    #    for ind in closest:
    #        print(f'label: {labels[ind]} \t similarity: {similarities[ind]}')

KeyboardInterrupt: 

In [6]:
from transformers import AutoTokenizer, AutoModel
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import pickle
from transformers import AutoModelForSequenceClassification, AdamW, AutoConfig
import pickle
from transformers import AutoTokenizer, AutoModel
from torch.nn import functional as F

accuracy_list= []
model_predicts = {}
#her 5 model için iterasyon kuruldu  'ytu-ce-cosmos/turkish-small-bert-uncased','ytu-ce-cosmos/turkish-mini-bert-uncased',
models= ['ytu-ce-cosmos/turkish-small-bert-uncased', 'ytu-ce-cosmos/turkish-mini-bert-uncased','ytu-ce-cosmos/turkish-medium-bert-uncased']


#tokenizer ve model burada tanımlanıyor
for model_name in models:

    name= model_name.split("/")[-1]
    tokenizer = AutoTokenizer.from_pretrained(model_name,do_lower_case=True)
    model = AutoModel.from_pretrained(model_name, num_labels=2)
    hidden_layer = model.config.num_hidden_layers
    print(f"Model has {hidden_layer} hidden layer.")


    #commentsin içerisindeki her satır için vektörler çıkartılıp cosine uzaklık predict_listin içerisine yazılıyor ardından bu liste model_predictsin içerisine yazılıyor. Ayrıca Accuracyler de Accuracy_list e yazılıyor


    labels = ["harika mükemmel bir film","berbat iğrenç bir film"]

    prediction_dict = {}

    predict_list = []
    for comment in comments:
        sentence= comment
            # run inputs through model and mean-pool over the sequence
        # dimension to get sequence-level representations
        inputs = tokenizer.batch_encode_plus([sentence] + labels,
                                            return_tensors='pt',
                                            pad_to_max_length=True)

        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        output = model(input_ids, attention_mask=attention_mask,output_hidden_states=True)[0]
        sentence_rep = output[:1].mean(dim=1)
        label_reps = output[1:].mean(dim=1)

        # now find the labels with the highest cosine similarities to
        # the sentence
        similarities = F.cosine_similarity(sentence_rep, label_reps)
        #closest en yakın olan labelı önce yazıyor closest[0] en yakını vermiş oluyor
        closest = similarities.argsort(descending=True)
        predict_list.append(closest[0])
    prediction_dict["pooled_layer"]= predict_list

    for layer_index in range(model.config.num_hidden_layers+1):
        print(layer_index)
        predict_list = []
        for comment in comments:
            sentence = comment
            # run inputs through model and mean-pool over the sequence
            # dimension to get sequence-level representations
            inputs = tokenizer.batch_encode_plus([sentence] + labels,
                                                return_tensors='pt',
                                                pad_to_max_length=True)

            input_ids = inputs['input_ids']
            attention_mask = inputs['attention_mask']
            output = model(input_ids, attention_mask=attention_mask,output_hidden_states=True)[2][layer_index]
            sentence_rep = output[:1].mean(dim=1)
            label_reps = output[1:].mean(dim=1)

            # now find the labels with the highest cosine similarities to
            # the sentence
            similarities = F.cosine_similarity(sentence_rep, label_reps)
            #closest en yakın olan labelı önce yazıyor closest[0] en yakını vermiş oluyor
            closest = similarities.argsort(descending=True)
            predict_list.append(closest[0])

        prediction_dict[layer_index] = predict_list

    with open(f'/content/drive/MyDrive/prediction_dict_{name}_etiket2.pickle', 'wb') as handle:
        pickle.dump(prediction_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/590 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/590 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/170M [00:00<?, ?B/s]

Model has 8 hidden layer.
0
1
2
3
4
5
6
7
8


In [ ]:
import pickle
with open('/content/drive/MyDrive/prediction_dict_etiket2.pickle', 'wb') as handle:
    pickle.dump(prediction_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle
with open('/content/drive/MyDrive/prediction_dict_2.pickle', 'wb') as handle:
    pickle.dump(prediction_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
prediction_dict

{'pooled_layer': [tensor(1),
  tensor(1),
  tensor(0),
  tensor(1),
  tensor(0),
  tensor(0),
  tensor(1),
  tensor(0),
  tensor(1),
  tensor(0),
  tensor(1),
  tensor(0),
  tensor(1),
  tensor(0),
  tensor(0),
  tensor(0),
  tensor(0),
  tensor(0),
  tensor(1),
  tensor(1),
  tensor(1),
  tensor(1),
  tensor(1),
  tensor(0),
  tensor(0),
  tensor(0),
  tensor(0),
  tensor(1),
  tensor(0),
  tensor(0),
  tensor(0),
  tensor(0),
  tensor(0),
  tensor(0),
  tensor(0),
  tensor(1),
  tensor(0),
  tensor(0),
  tensor(1),
  tensor(0),
  tensor(1),
  tensor(0),
  tensor(0),
  tensor(1),
  tensor(0),
  tensor(1),
  tensor(0),
  tensor(1),
  tensor(0),
  tensor(0),
  tensor(1),
  tensor(1),
  tensor(0),
  tensor(0),
  tensor(0),
  tensor(1),
  tensor(0),
  tensor(1),
  tensor(1),
  tensor(1),
  tensor(0),
  tensor(1),
  tensor(0),
  tensor(0),
  tensor(0),
  tensor(0),
  tensor(1),
  tensor(0),
  tensor(0),
  tensor(1),
  tensor(1),
  tensor(0),
  tensor(0),
  tensor(0),
  tensor(1),
  tensor(

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
for a,b in prediction_dict.items():
    accuracy_score_1 = accuracy_score(labels_test, b)
    accuracy_list.append(accuracy_score_1)
    print(f"Accuracy of BERT ({a}) is:",accuracy_score_1)


    print(classification_report(labels_test, b))

Accuracy of BERT (pooled_layer) is: 0.715
              precision    recall  f1-score   support

           0       0.65      0.91      0.76       500
           1       0.85      0.52      0.65       500

    accuracy                           0.71      1000
   macro avg       0.75      0.72      0.70      1000
weighted avg       0.75      0.71      0.70      1000

Accuracy of BERT (0) is: 0.527
              precision    recall  f1-score   support

           0       0.83      0.07      0.13       500
           1       0.51      0.99      0.68       500

    accuracy                           0.53      1000
   macro avg       0.67      0.53      0.40      1000
weighted avg       0.67      0.53      0.40      1000

Accuracy of BERT (1) is: 0.565
              precision    recall  f1-score   support

           0       0.73      0.21      0.32       500
           1       0.54      0.92      0.68       500

    accuracy                           0.56      1000
   macro avg       0.63 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

              precision    recall  f1-score   support

           0       0.50      1.00      0.67       500
           1       0.00      0.00      0.00       500

    accuracy                           0.50      1000
   macro avg       0.25      0.50      0.33      1000
weighted avg       0.25      0.50      0.33      1000

Accuracy of BERT (10) is: 0.5
              precision    recall  f1-score   support

           0       0.50      1.00      0.67       500
           1       0.00      0.00      0.00       500

    accuracy                           0.50      1000
   macro avg       0.25      0.50      0.33      1000
weighted avg       0.25      0.50      0.33      1000

Accuracy of BERT (11) is: 0.513
              precision    recall  f1-score   support

           0       0.51      1.00      0.67       500
           1       1.00      0.03      0.05       500

    accuracy                           0.51      1000
   macro avg       0.75      0.51      0.36      1000
weighted avg